In [ ]:
# importing required packages
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
# from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import cohen_kappa_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

In [ ]:
import csv

In [ ]:
importing_dataset = pd.read_csv('https://github.com/dnyanada02/SmartGrading/blob/main/Dataset/training_set_rel3.tsv?raw=true', quoting=csv.QUOTE_NONE, sep='\t', encoding='ISO-8859-1')
# dependent variable
scores = importing_dataset['domain1_score']
dataset = importing_dataset.loc[:,['essay_id', 'essay_set', 'essay', 'domain1_score']]
dataset.dropna()
dataset

# dataset = pd.read_csv("/content/training_set_rel3.tsv",sep='\t', encoding='ISO-8859-1',
#                             usecols = ['essay_id', 'essay_set', 'essay','domain1_score']).dropna(axis=1)
# scores = dataset['domain1_score']
# dataset

,essay_id,essay_set,essay,domain1_score
0,1,1,"""Dear local newspaper, I think effects compute...",8
1,2,1,"""Dear @CAPS1 @CAPS2, I believe that using comp...",9
2,3,1,"""Dear, @CAPS1 @CAPS2 @CAPS3 More and more peop...",7
3,4,1,"""Dear Local Newspaper, @CAPS1 I have found tha...",10
4,5,1,"""Dear @LOCATION1, I know having computers has ...",8
...,...,...,...,...
12973,21626,8,""" In most stories mothers and daughters are ei...",35
12974,21628,8,""" I never understood the meaning laughter is t...",32
12975,21629,8,"""When you laugh, is @CAPS5 out of habit, or is...",40
12976,21630,8,""" Trippin' on fe...",40


In [ ]:
dataset.describe()

,essay_id,essay_set,domain1_score
count,12978.000000,12978.000000,12978.000000
mean,10295.432809,4.179458,6.799276
std,6308.588616,2.136749,8.970357
min,1.000000,1.000000,0.000000
25%,4439.250000,2.000000,2.000000
50%,10045.500000,4.000000,3.000000
75%,15680.750000,6.000000,8.000000
max,21633.000000,8.000000,60.000000


In [ ]:
# Generating word tokens after removing characters other than alphabets, converting them to lower case and
# removing stopwords from the text'''

def word_tokens(essay_text):
    essay_text = re.sub("[^a-zA-Z]", " ", essay_text)
    words = essay_text.lower().split()
    stop_words = set(stopwords.words("english"))
    words = [w for w in words if not w in stop_words]
    return (words)

In [ ]:
# Generating sentence tokens from the essay and finally the word tokens

def sentence_tokens(essay_text):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sent_tokens = tokenizer.tokenize(essay_text.strip())
    sentences = []
    for sent_token in sent_tokens:
        if len(sent_token) > 0:
            sentences.append(word_tokens(sent_token))
    return sentences

In [ ]:
# Generating a vector of features

def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

In [ ]:
# Generating word vectors to be used in word2vec model

def getAvgFeatureVecs(essays, model, num_features):
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay_text in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay_text, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [ ]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [ ]:
X=dataset.copy()

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Applying k-fold cross validation

cv = KFold(n_splits=2, shuffle=True)
cv.get_n_splits(len(dataset))
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv.split(dataset):
    print("\n------------Fold {}------------\n".format(count))
    X_test, X_train, y_test, y_train = dataset.iloc[testcv], dataset.iloc[traincv], scores.iloc[testcv], scores.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    for essay in train_essays:
            # Obtaining all sentences from the training set of essays.
            sentences += sentence_tokens(essay)
            
    # Initializing variables for word2vec model.
    num_features = 300 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

    clean_train_essays = []
    
    # Generate training and testing data word vectors.
    for essay_text in train_essays:
        clean_train_essays.append(word_tokens(essay_text))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_text in test_essays:
        clean_test_essays.append(word_tokens(essay_text))
    testDataVecs = getAvgFeatureVecs(clean_test_essays, model, num_features)
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    trainDataVecs = np.nan_to_num(trainDataVecs.astype(np.float32))
    testDataVecs = np.nan_to_num(testDataVecs.astype(np.float32))

    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(testDataVecs)
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    '''Evaluation metric used : 
    1. Mean squared error
    2. Variance
    3. Cohen's kappa score
    Expected results - Minimum error, maximum variance(For variance, best possible score is 1.0, lower 
    values are worse.) and maximum kappa score(1 depicting the best scores)'''
    
    # Mean squared error
    print("Mean squared error: {0:.2f}".format(mean_squared_error(y_test.values, y_pred)))

    # Explained variance score: 1 is perfect prediction
    print('Variance: {0:.2f}'.format(explained_variance_score(y_test.values, y_pred)))  
    
    #Cohen's kappa score
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {0:.2f}".format(result))
    results.append(result)

    count += 1


------------Fold 1------------

Training Word2Vec Model...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 300)            721200    
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
102/102 [==============================] - 8s 27ms/step - loss: 72.5976 - mae: 4.7499
Epoch 2/50
102/102 [==============================] - 3s 28ms/step - loss: 48.7990 - 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 1, 300)            721200    
                                                                 
 lstm_3 (LSTM)               (None, 64)                93440     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
102/102 [==============================] - 8s 27ms/step - loss: 75.0303 - mae: 4.7608
Epoch 2/50
102/102 [==============================] - 3s 27ms/step - loss: 49.4874 

In [ ]:
print("Average Kappa score after a 2-fold cross validation: ",np.around(np.array(results).mean(),decimals=2))

Average Kappa score after a 2-fold cross validation:  0.95


# Other Models


In [ ]:
# Splitting dataset into training and test set and generating word embeddings for other models other than
# neural networks

indep_train, indep_test, dep_train, dep_test = train_test_split(dataset, scores, test_size = 0.25)
train_essays2 = indep_train['essay']
test_essays2 = indep_test['essay']
    
sentences2 = []


for essay2 in train_essays2:
            # Obtaining all sentences from the training set of essays.
            sentences2 += sentence_tokens(essay2)
            
# Initializing variables for word2vec model.
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

print("Training Word2Vec Model...")
model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

clean_train_essays2 = []
    
# Generate training and testing data word vectors.
for essay_text2 in train_essays2:
    clean_train_essays2.append(word_tokens(essay_text2))
trainDataVecs2 = getAvgFeatureVecs(clean_train_essays2, model, num_features)
    
clean_test_essays2 = []
for essay_text2 in test_essays2:
    clean_test_essays2.append(word_tokens(essay_text2))
testDataVecs2 = getAvgFeatureVecs(clean_test_essays2, model, num_features)
    
trainDataVecs2 = np.array(trainDataVecs2)
testDataVecs2 = np.array(testDataVecs2)
trainDataVecs2 = np.nan_to_num(trainDataVecs2.astype(np.float32))
testDataVecs2 = np.nan_to_num(testDataVecs2.astype(np.float32))
# trainDataVecs2 = np.any(np.isnan(trainDataVecs2))
# testDataVecs2 = np.any(np.isnan(testDataVecs2))
# trainDataVecs2[np.isfinite(trainDataVecs2) == True] = 0
# testDataVecs2[np.isfinite(testDataVecs2) == True] = 0
# dep_train[np.isfinite(dep_train) == True] = 0
# trainDataVecs2 = np.isnan(trainDataVecs2.any()) #and gets False
# trainDataVecs2 = np.isfinite(trainDataVecs2.all())
# testDataVecs2 = np.isnan(testDataVecs2.any()) #and gets False
# testDataVecs2 = np.isfinite(testDataVecs2.all()) 

Training Word2Vec Model...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# Generating scores using Linear Regression Model

linear_regressor = LinearRegression()

linear_regressor.fit(trainDataVecs2, dep_train)

dep_pred = linear_regressor.predict(testDataVecs2)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(dep_test, dep_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % explained_variance_score(dep_test,dep_pred))

#print('Cohen\'s kappa score: %.2f' % cohen_kappa_score(dep_pred, dep_test))
print("Kappa Score: {0:.2f}".format(cohen_kappa_score(dep_test.values,np.around(dep_pred),weights='quadratic')))

Mean squared error: 20.94
Variance score: 0.73
Kappa Score: 0.85


In [ ]:
# Generating scores using K-Nearest Neighbor
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(trainDataVecs2, dep_train)
dep_pred = knn.predict(testDataVecs2)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(dep_test, dep_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % explained_variance_score(dep_test,dep_pred))

#print('Cohen\'s kappa score: %.2f' % cohen_kappa_score(dep_pred, dep_test))
print("Kappa Score: {0:.2f}".format(cohen_kappa_score(dep_test.values,np.around(dep_pred),weights='quadratic')))


Mean squared error: 14.70
Variance score: 0.82
Kappa Score: 0.92


In [ ]:
 #Generating scores using Desion Tree Classifier
dt = DecisionTreeClassifier(criterion = 'entropy',random_state=0,max_depth = 30)
dt.fit(trainDataVecs2, dep_train)
dep_pred = dt.predict(testDataVecs2)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(dep_test, dep_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % explained_variance_score(dep_test,dep_pred))

#print('Cohen\'s kappa score: %.2f' % cohen_kappa_score(dep_pred, dep_test))
print("Kappa Score: {0:.2f}".format(cohen_kappa_score(dep_test.values,np.around(dep_pred),weights='quadratic')))

Mean squared error: 22.56
Variance score: 0.71
Kappa Score: 0.86


In [ ]:
svr = SVR()

'''parameters = {'kernel':['linear', 'rbf'], 'C':[1, 100], 'gamma':[0.1, 0.001]}

grid = GridSearchCV(svr, parameters)
grid.fit(trainDataVecs2, dep_train)

y_pred = grid.predict(testDataVecs2)

# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_)'''

#USING THE PARAMS FOUND OUT USING GRID SEARCH CV
svr = SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
svr.fit(trainDataVecs2, dep_train)
dep_pred = svr.predict(testDataVecs2)


# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(dep_test, dep_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % explained_variance_score(dep_test,dep_pred))

#Cohen's Kappa score
print("Kappa Score: {0:.2f}".format(cohen_kappa_score(dep_test.values,np.around(dep_pred),weights='quadratic')))

Mean squared error: 38.19
Variance score: 0.53
Kappa Score: 0.64
